# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [35]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [36]:
df_eng = pd.read_csv("data/movie_reviews_eng.csv")

In [37]:
df_eng.sample(10)

,Movie / TV Series,Year,Synopsis,Review
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and..."
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su..."
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale..."
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ..."
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina..."
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of..."
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun..."
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi..."
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ..."
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio..."


In [38]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng['Original_Language'] = 'English'

    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr['Original_Language'] = 'French'

    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp['Original_Language'] = 'Spanish'

    #Ensures that all DataFrames have the same column names
    df_fr.columns = df_eng.columns
    df_sp.columns = df_eng.columns

    #Combines the three DataFrames (df_eng, df_French, and df_Spanish) into a single DataFrame named df.
    #Resets the index of the combined DataFrame to avoid issues with duplicate indices.
    df = pd.concat([df_eng, df_fr, df_sp]).reset_index(drop='True')

    df.rename(columns={'Movie / TV Series': 'Title'}, inplace=True)
    return df

df = preprocess_data()

In [39]:
df.sample(10)

,Title,Year,Synopsis,Review,Original_Language
26,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",Spanish
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",Spanish
11,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
23,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",Spanish
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
24,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",Spanish


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [48]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

In [49]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews =  df[df.Original_Language == 'French']['Review']
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, model=fr_en_model, tokenizer=fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df[df.Original_Language == 'French']['Synopsis']
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, model=fr_en_model, tokenizer=fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[df.Original_Language == 'Spanish']['Review']
es_reviews_en = es_reviews.apply(lambda x: translate(x, model=es_en_model, tokenizer=es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[df.Original_Language == 'Spanish']['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, model=es_en_model, tokenizer=es_en_tokenizer))

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

df.loc[df.Original_Language == 'French', 'Review'] = fr_reviews_en
df.loc[df.Original_Language == 'French', 'Synopsis'] = fr_synopsis_en
df.loc[df.Original_Language == 'Spanish', 'Review'] = es_reviews_en
df.loc[df.Original_Language == 'Spanish', 'Synopsis'] = es_synopsis_en

In [50]:
df.sample(10)

,Title,Year,Synopsis,Review,Original_Language
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",French
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",Spanish
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",French
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [54]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)[0]
    sentiment = "Positive" if result['label'] == 'POSITIVE' else "Negative"

    return sentiment

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [55]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))


In [56]:
df.sample(10)

,Title,Year,Synopsis,Review,Original_Language,Sentiment
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish,Negative
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French,Positive
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,Positive
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English,Positive
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Spanish,Negative
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English,Negative
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",French,Positive
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English,Negative
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French,Negative
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",French,Negative


In [63]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)